# NB-HI4-1: Long-term annual total fluxes changes for early, mid and late 21st century periods from historical period

## module loading...

In [ ]:
%matplotlib inline  
import os
import sys
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from timeit import default_timer as timer

print("\nThe Python version: %s.%s.%s" % sys.version_info[:3])
print(xr.__name__, xr.__version__)

mpl.rcParams['xtick.labelsize']='small'

In [ ]:
from cmip5_oconus import print_date
print_date()

from cmip5_oconus.data_catalog import load_monthly_historical_hydro_datasets, resample_data
from cmip5_oconus.data_catalog import load_monthly_cmip5_hydro_datasets
from cmip5_oconus.plot import custom_div_cmap, MidpointNormalize, add_ylabel
from cmip5_oconus.utils import calc_change

In [ ]:
from dask_jobqueue import PBSCluster
cluster = PBSCluster(processes=9, threads=4, memory="108GB",queue='casper', 
                     walltime='00:40:00')
cluster.scale(jobs=3)

In [ ]:
from dask.distributed import Client
client = Client(cluster)

In [ ]:
client

-------------------------
## Setup 

In [ ]:
figSave = True

In [ ]:
region          = 'HI'
gcms            = ["ACCESS1-3","CanESM2","CCSM4","CSIRO-Mk3-6-0","GFDL-ESM2M","HadGEM2-ES","inmcm4","MIROC5","MPI-ESM-MR","MRI-CGCM3"]
hydro_flux_vars = ['PRCP', 'EVAP', 'total_runoff']
period_hist     = slice('1970-01-01', '1999-12-31')
period_futr     = {'early' : slice('2010-01-01', '2039-12-31'),
                   'middle': slice('2040-01-01', '2069-12-31'),
                   'late'  : slice('2070-01-01', '2099-12-31')}

## Load the data

In [ ]:
xr.set_options(file_cache_maxsize=12000)
histo_data = load_monthly_historical_hydro_datasets(models=gcms, variables=hydro_flux_vars, region=region, parallel=False)
rcp45_data = load_monthly_cmip5_hydro_datasets(scen='rcp45', models=gcms, variables=hydro_flux_vars, region=region, parallel=False)
rcp85_data = load_monthly_cmip5_hydro_datasets(scen='rcp85', models=gcms, variables=hydro_flux_vars, region=region, parallel=False)

## Compute 30-yr annual mean for 4 periods - historical, early, mid and late

In [ ]:
%%time
histo_hydro_means = resample_data(histo_data['gcm'].sel(time=period_hist), region=region, freq='AS').mean('time').compute()

rcp45_hydro_means = {}
rcp85_hydro_means = {}
for key, futr_period in period_futr.items():
    rcp45_hydro_means[key] = resample_data(rcp45_data.sel(time=futr_period), region=region, freq='AS').mean('time').compute()
    rcp85_hydro_means[key] = resample_data(rcp85_data.sel(time=futr_period), region=region, freq='AS').mean('time').compute()

## Compute 30-yr annual runoff ratio and  Evaporation ratio

In [ ]:
%%time
histo_hydro_means['er'] = histo_hydro_means['EVAP']/histo_hydro_means['PRCP']
histo_hydro_means['rr']   = histo_hydro_means['total_runoff']/histo_hydro_means['PRCP']
for key, futr_period in period_futr.items():
    rcp45_hydro_means[key]['er'] = rcp45_hydro_means[key]['EVAP']/rcp45_hydro_means[key]['PRCP']
    rcp45_hydro_means[key]['rr'] = rcp45_hydro_means[key]['total_runoff']/rcp45_hydro_means[key]['PRCP']
    
    rcp85_hydro_means[key]['er'] = rcp85_hydro_means[key]['EVAP']/rcp85_hydro_means[key]['PRCP']
    rcp85_hydro_means[key]['rr'] = rcp85_hydro_means[key]['total_runoff']/rcp85_hydro_means[key]['PRCP']    

## Compute difference in 30-yr annual mean between historical and future

In [ ]:
# future change from historical [mm] 
rcp45_hydro_30yr_diff = {}
rcp85_hydro_30yr_diff = {}
diff_rcp_hydro_30yr_change ={}
for key, futr_period in period_futr.items():
    rcp45_hydro_30yr_diff[key] = calc_change(histo_hydro_means, rcp45_hydro_means[key], pct=False).persist()
    rcp85_hydro_30yr_diff[key] = calc_change(histo_hydro_means, rcp85_hydro_means[key], pct=False).persist()

    diff_rcp_hydro_30yr_change[key] =rcp85_hydro_30yr_diff[key] - rcp45_hydro_30yr_diff[key]

## Plotting

In [ ]:
# create colormap
# ---------------
from matplotlib.colors import LinearSegmentedColormap
# create a colormap that consists of
# - 1/5 : custom colormap, ranging from white to the first color of the colormap
# - 4/5 : existing colormap

# set upper part: 4 * 256/4 entries
upper = mpl.cm.Spectral_r(np.arange(256))

# set lower part: 1 * 256/4 entries
# - initialize all entries to 1 to make sure that the alpha channel (4th column) is 1
lower = np.ones((int(256/4),4))
# - modify the first three columns (RGB):
#   range linearly between white (1,1,1) and the first color of the upper colormap
for i in range(3):
    lower[:,i] = np.linspace(0.9, upper[0,i], lower.shape[0])

# combine parts of colormap
cmap = np.vstack(( lower, upper ))

# convert to matplotlib colormap
cmap = mpl.colors.ListedColormap(cmap, name='myColorMap', N=cmap.shape[0])

# option1
cmap1 = mpl.cm.YlGnBu
cmap1.set_under('xkcd:light grey')


# option 2
cmap2 = LinearSegmentedColormap.from_list('custom1', 
                                          [(0.00, 'xkcd:very light green'),
                                           (0.20, 'xkcd:light blue'),
                                           (0.50, 'xkcd:lightish blue'),
                                           (0.70, 'xkcd:blue'),
                                           (1.00, 'xkcd:purple')], N=256)
cmap2.set_over('xkcd:dark purple')
cmap2.set_under('xkcd:light grey')

------------
 - Change in 30yr mean annual total fluxes between historical and three future periods in 21st century (mid, and late) 

In [ ]:
cm = {'history'   : cmap2,
      'change_PR' : custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'change_ET' : custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'change_RO' : custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),}

sfmt = mpl.ticker.ScalarFormatter(useMathText=True)
sfmt.set_powerlimits((-5, 5))

cbar_kwrgs = {'mean':   {"orientation":"horizontal", "shrink":0.990, "pad":0.03, 'format':sfmt},
              'change': {"orientation":"horizontal", "shrink":0.990, "pad":0.03}}

style_kwargs = {'mean':   {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': cbar_kwrgs['mean']},
                'change': {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': cbar_kwrgs['change']}
               }
#'PRCP':                {'levels': 9, 'vmin':0, 'vmax':4000, 'cmap':cm['history'], 'extend':'max',  **style_kwargs['mean']}, #'levels': 41,
kwargs = {'change-PRCP':         {'levels': 9, 'vmin':-100, 'vmax':100, 'cmap':cm['change_PR'], 'extend':'both', **style_kwargs['change']},
          'change-EVAP':         {'levels': 9, 'vmin':-100, 'vmax':100, 'cmap':cm['change_ET'], 'extend':'both', **style_kwargs['change']},
          'change-total_runoff': {'levels': 9, 'vmin':-100, 'vmax':100, 'cmap':cm['change_RO'], 'extend':'both', **style_kwargs['change']},
          'PRCP':                {'levels': 21, 'vmin':0, 'vmax':4000, 'cmap':cm['history'], 'extend':'max',  **style_kwargs['mean']}, #'levels': 41, 
          'EVAP':                {'levels': 11,'vmin':0, 'vmax':1000,  'cmap':cm['history'], 'extend':'both', **style_kwargs['mean']}, #'levels': 46, 
          'total_runoff':        {'levels': 21, 'vmin':0, 'vmax':4000, 'cmap':cm['history'], 'extend':'max',  **style_kwargs['mean']}} #'levels': 11, 

var_title = {'PRCP':'Precipitation', 'EVAP':'Evapotranspiration', 'total_runoff':'Runoff'}

fig, axes = plt.subplots(nrows=3, ncols=3, sharex=True, sharey=True, figsize=(8.125,8.75))
plt.subplots_adjust(left=0.045, bottom=0.0125, right=0.985, top=0.975, hspace=0.075, wspace=0.075)

for col, var in enumerate(['PRCP','EVAP','total_runoff']):
    histo_hydro_means.mean(dim='gcm')[var].plot.pcolormesh(ax=axes[0, col], **kwargs[var])
    rcp85_hydro_30yr_diff['middle'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[1, col], **kwargs[f'change-{var}'])
    rcp85_hydro_30yr_diff['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[2, col],   **kwargs[f'change-{var}'])
    
    add_ylabel(axes[0, 0], 'historical [mm/yr]', fontsize=11)
    add_ylabel(axes[1, 0], 'changes 2040-2069 [mm/yr]', fontsize=11)
    add_ylabel(axes[2, 0], 'changes 2070-2099 [mm/yr]', fontsize=11)
    axes[0, col].set_title(var_title[var], fontsize=12)
    
#plt.tight_layout()
if figSave:
    plt.savefig(f'NB-HI4_Fig1_annual_mean_change_mid_late_rcp85.png', dpi=300)

## Runnoff ratio and evaporation ratio - historical ensemble mean and changes

In [ ]:
cmap_change_ratio = custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue')
cmap_change_ratio.set_over('xkcd:blue blue')
cmap_change_ratio.set_under('xkcd:magenta')

cm = {'history'   : cmap2,
      'change_er' : cmap_change_ratio,
      'change_rr' : cmap_change_ratio}

sfmt = mpl.ticker.ScalarFormatter(useMathText=True)
sfmt.set_powerlimits((-5, 5))

cbar_kwrgs = {'mean':   {"orientation":"horizontal", "shrink":0.990, "pad":0.03, 'format':sfmt},
              'change': {"orientation":"horizontal", "shrink":0.990, "pad":0.03}}

style_kwargs = {'mean':   {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': cbar_kwrgs['mean']},
                'change': {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': cbar_kwrgs['change']}
               }

kwargs = {'change-er':  {'levels': 11, 'vmin':-0.05, 'vmax':0.05, 'cmap':cm['change_er'], 'extend':'both',    **style_kwargs['change']},
          'change-rr':  {'levels': 11, 'vmin':-0.05, 'vmax':0.05, 'cmap':cm['change_rr'], 'extend':'both',    **style_kwargs['change']},
          'er':         {'levels': 11, 'vmin':0.0, 'vmax':1.0,  'cmap':cm['history'],   'extend':'neither', **style_kwargs['mean']}, #'levels': 46, 
          'rr':         {'levels': 11, 'vmin':0.0, 'vmax':1.0,  'cmap':cm['history'],   'extend':'neither', **style_kwargs['mean']}} #'levels': 11, 

var_title = {'er':'Evaporation Efficiency', 'rr':'Runoff Efficiency'}

fig, axes = plt.subplots(nrows=3, ncols=2, sharex=True, sharey=True, figsize=(6.50,8.75))
plt.subplots_adjust(left=0.045, bottom=0.0125, right=0.985, top=0.975, hspace=0.075, wspace=0.125)

for col, var in enumerate(['er','rr']):
    histo_hydro_means.mean(dim='gcm')[var].plot.pcolormesh(ax=axes[0, col], **kwargs[var])
    rcp85_hydro_30yr_diff['middle'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[1, col], **kwargs[f'change-{var}'])
    rcp85_hydro_30yr_diff['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[2, col],   **kwargs[f'change-{var}'])
    
    add_ylabel(axes[0, 0], 'historical [-]', fontsize=11)
    add_ylabel(axes[1, 0], 'changes 2040-2069 [-]', fontsize=11)
    add_ylabel(axes[2, 0], 'changes 2070-2099 [-]', fontsize=11)
    axes[0, col].set_title(var_title[var], fontsize=12)
    
#plt.tight_layout()
if figSave:
    plt.savefig(f'NB-HI4_Fig13_annual_mean_change_efficiency_mid_late_rcp85.png', dpi=300)

------------
 - Difference between two RCPs in change in 30yr mean annual total for late 21st century

In [ ]:
cm = {'change' : custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue')}

style_kwargs = {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.995, pad=0.03)}

kwargs       = {'change-rcp_PRCP':         {'levels':9, 'vmin':-200, 'vmax':200,  'cmap':cm['change'], 'extend':'both', **style_kwargs},
                'change-rcp_EVAP':         {'levels':9, 'vmin':-100, 'vmax':100,  'cmap':cm['change'], 'extend':'both', **style_kwargs},
                'change-rcp_total_runoff': {'levels':9, 'vmin':-200, 'vmax':200,  'cmap':cm['change'], 'extend':'both', **style_kwargs},
               }

var_title = {'PRCP':'Precipitation', 'EVAP':'Evapotranspiration', 'total_runoff':'Runoff'}

fig, axes = plt.subplots(nrows=1, ncols=3, sharex=True, sharey=True, figsize=(8.125,3))
plt.subplots_adjust(left=0.040, bottom=0.0125, right=0.990, top=0.900, hspace=0.060, wspace=0.060)

for col, var in enumerate(['PRCP','EVAP','total_runoff']):
    diff_rcp_hydro_30yr_change['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[col], **kwargs[f'change-rcp_{var}'])
    
    add_ylabel(axes[0], 'RCP85 - RCP45[mm/yr]', fontsize=11)
    axes[col].set_title(var_title[var], fontsize=12)
    
if figSave:
    plt.savefig(f'NB-HI4_Fig2_annual_mean_change_late_diff_rcp.png', dpi=300)

--------
- Climate changes for individual GCMs for late 21st century are plotted below

--------------------
### Precipitation

#### rcp45

In [ ]:
var="PRCP"

cm = {'mean': custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'std' : custom_div_cmap(numcolors=255, mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.975, pad=0.03)}

kwargs = {'mean': {'levels': 7, 'vmin':-300, 'vmax':300, 'cmap':cm['mean'], 'extend':'both', **style_kwargs},
          'std':  {'levels': 11, 'vmin':0,    'vmax':500, 'cmap':cm['std'],  'extend':'max',  **style_kwargs}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5)) # used to be figsize=(14,11)
plt.subplots_adjust(left=0.0125, bottom=0.0125, right=0.99, top=0.935, hspace=0.075, wspace=0.075)

for ix, gcm in enumerate(gcms+['ensemble-mean','ensemble-std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        rcp45_hydro_30yr_diff['late'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 10:
        rcp45_hydro_30yr_diff['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 11:
        rcp45_hydro_30yr_diff['late'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=12)
    
#plt.tight_layout()
fig.suptitle('Change in 30yr mean annual precipitation [mm/yr] - late 21st century', fontsize=14, y=0.985)

if figSave:
    plt.savefig(f'NB-HI4_Fig3_annual_mean_change_late_{var}_rcp45.png', dpi=300)

#### rcp85

In [ ]:
var="PRCP"

cm = {'mean': custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'std' : custom_div_cmap(numcolors=255, mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.975, pad=0.03)}

kwargs = {'mean': {'levels': 7, 'vmin':-300, 'vmax':300, 'cmap':cm['mean'], 'extend':'both', **style_kwargs},
          'std':  {'levels': 11, 'vmin':0,    'vmax':500, 'cmap':cm['std'],  'extend':'max',  **style_kwargs}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5))  # used to be figsize=(14,11)
plt.subplots_adjust(left=0.0125, bottom=0.0125, right=0.99, top=0.935, hspace=0.075, wspace=0.075)

for ix, gcm in enumerate(gcms+['ensemble-mean','ensemble-std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        rcp85_hydro_30yr_diff['late'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 10:
        rcp85_hydro_30yr_diff['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 11:
        rcp85_hydro_30yr_diff['late'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=12)
    
fig.suptitle('Change in 30yr mean annual precipitation [mm/yr]- late 21st century', fontsize=14, y=0.985)

if figSave:
    plt.savefig(f'NB-HI4_Fig4_annual_mean_change_late_{var}_rcp85.png', dpi=300)

In [ ]:
var="PRCP"

cm = {'mean': custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'std' : custom_div_cmap(numcolors=255, mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.975, pad=0.03)}

kwargs = {'mean': {'levels': 7, 'vmin':-300, 'vmax':300, 'cmap':cm['mean'], 'extend':'both', **style_kwargs},
          'std':  {'levels': 11, 'vmin':0,   'vmax':500, 'cmap':cm['std'],  'extend':'max',  **style_kwargs}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5))  # used to be figsize=(14,11)
plt.subplots_adjust(left=0.0125, bottom=0.0125, right=0.99, top=0.935, hspace=0.075, wspace=0.075)

for ix, gcm in enumerate(gcms+['ensemble-mean','ensemble-std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        diff_rcp_hydro_30yr_change['late'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 10:
        diff_rcp_hydro_30yr_change['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 11:
        diff_rcp_hydro_30yr_change['late'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=14)
    
fig.suptitle('RCP85 vs RCP 45: Change in 30yr mean annual precipitation [mm/yr] - late 21st century', fontsize=14, y=0.985)

if figSave:
    plt.savefig(f'NB-HI4_Fig5_annual_mean_change_late_{var}_rcp_diff.png', dpi=300)

--------------------
### total runoff

#### rcp45

In [ ]:
var="total_runoff"

cm = {'mean': custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'std' : custom_div_cmap(numcolors=255, mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.975, pad=0.03)}

kwargs = {'mean': {'levels': 7, 'vmin':-300, 'vmax':300, 'cmap':cm['mean'], 'extend':'both', **style_kwargs},
          'std':  {'levels': 11, 'vmin':0,    'vmax':500, 'cmap':cm['std'],  'extend':'max',  **style_kwargs}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5)) # used to be figsize=(14,11)
plt.subplots_adjust(left=0.0125, bottom=0.0125, right=0.99, top=0.935, hspace=0.075, wspace=0.075)

for ix, gcm in enumerate(gcms+['ensemble-mean','ensemble-std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        rcp45_hydro_30yr_diff['late'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 10:
        rcp45_hydro_30yr_diff['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 11:
        rcp45_hydro_30yr_diff['late'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=12)
    
fig.suptitle('Change in 30yr mean annual total runoff [mm/yr] - late 21st century', fontsize=14, y=0.985)

if figSave:
    plt.savefig(f'NB-HI4_Fig6_annual_mean_change_late_{var}_rcp45.png', dpi=300)

#### rcp85

In [ ]:
var="total_runoff"

cm = {'mean': custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'std' : custom_div_cmap(numcolors=255, mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.975, pad=0.03)}

kwargs = {'mean': {'levels': 7, 'vmin':-300, 'vmax':300, 'cmap':cm['mean'], 'extend':'both', **style_kwargs},
          'std':  {'levels': 11, 'vmin':0,    'vmax':500, 'cmap':cm['std'],  'extend':'max',  **style_kwargs}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5))  # used to be figsize=(14,11)
plt.subplots_adjust(left=0.0125, bottom=0.0125, right=0.99, top=0.935, hspace=0.075, wspace=0.075)

for ix, gcm in enumerate(gcms+['ensemble-mean','ensemble-std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        rcp85_hydro_30yr_diff['late'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 10:
        rcp85_hydro_30yr_diff['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 11:
        rcp85_hydro_30yr_diff['late'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=12)
    
fig.suptitle('Change in 30yr mean annual total runoff [mm/yr] - late 21st century', fontsize=14, y=0.985)

if figSave:
    plt.savefig(f'NB-HI4_Fig7_annual_mean_change_late_{var}_rcp85.png', dpi=300)

#### rcp85 - rcp45

In [ ]:
var="total_runoff"

cm = {'mean': custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'std' : custom_div_cmap(numcolors=255, mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.975, pad=0.03)}

kwargs = {'mean': {'levels': 7, 'vmin':-300, 'vmax':300, 'cmap':cm['mean'], 'extend':'both', **style_kwargs},
          'std':  {'levels': 11, 'vmin':0,    'vmax':500, 'cmap':cm['std'],  'extend':'max',  **style_kwargs}}
fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5))  # used to be figsize=(14,11)
plt.subplots_adjust(left=0.0125, bottom=0.0125, right=0.99, top=0.935, hspace=0.075, wspace=0.075)

for ix, gcm in enumerate(gcms+['ensemble-mean','ensemble-std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        diff_rcp_hydro_30yr_change['late'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 10:
        diff_rcp_hydro_30yr_change['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 11:
        diff_rcp_hydro_30yr_change['late'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=14)
    
fig.suptitle('RCP85 vs RCP 45: Change in 30yr mean annual total runoff [mm/yr] - late 21st century', fontsize=14, y=0.985)

if figSave:
    plt.savefig(f'NB-HI4_Fig8_annual_mean_change_late_{var}_rcp_diff.png', dpi=300)

-------------------------
### Evapotranspiration

#### rcp45

In [ ]:
var="EVAP"

cm = {'mean': custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'std' : custom_div_cmap(numcolors=255, mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.975, pad=0.03)}

kwargs = {'mean': {'levels': 9, 'vmin':-100, 'vmax':100, 'cmap':cm['mean'], 'extend':'both', **style_kwargs},
          'std':  {'levels': 11, 'vmin':0,    'vmax':50, 'cmap':cm['std'],  'extend':'max',  **style_kwargs}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5))
plt.subplots_adjust(left=0.0125, bottom=0.0125, right=0.99, top=0.935, hspace=0.075, wspace=0.075)

for ix, gcm in enumerate(gcms+['ensemble-mean','ensemble-std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        rcp45_hydro_30yr_diff['late'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 10:
        rcp45_hydro_30yr_diff['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 11:
        rcp45_hydro_30yr_diff['late'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=12)
    
fig.suptitle('Change in 30yr mean annual evapotranspiration [mm/yr] - late 21st century', fontsize=14, y=0.985)

if figSave:
    plt.savefig(f'NB-HI4_Fig9_annual_mean_change_late_{var}_rcp45.png', dpi=300)

#### rcp85

In [ ]:
var="EVAP"

cm = {'mean': custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'std' : custom_div_cmap(numcolors=255, mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.975, pad=0.03)}

kwargs = {'mean': {'levels': 9, 'vmin':-100, 'vmax':100, 'cmap':cm['mean'], 'extend':'both', **style_kwargs},
          'std':  {'levels': 11, 'vmin':0,    'vmax':50, 'cmap':cm['std'],  'extend':'max',  **style_kwargs}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5))
plt.subplots_adjust(left=0.0125, bottom=0.0125, right=0.99, top=0.935, hspace=0.075, wspace=0.075)

for ix, gcm in enumerate(gcms+['ensemble-mean','ensemble-std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        rcp85_hydro_30yr_diff['late'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 10:
        rcp85_hydro_30yr_diff['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 11:
        rcp85_hydro_30yr_diff['late'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=12)
    
fig.suptitle('Change in 30yr mean annual evapotransipiration [mm/yr] - late 21st century', fontsize=14, y=0.985)

if figSave:
    plt.savefig(f'NB-HI4_Fig10_annual_mean_change_late_{var}_rcp85.png', dpi=300)

#### RCP85 - RCP45

In [ ]:
var="EVAP"

cm = {'mean': custom_div_cmap(numcolors=255, mincol='xkcd:red', midcol='xkcd:light grey', maxcol='xkcd:blue'),
      'std' : custom_div_cmap(numcolors=255, mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", shrink=0.975, pad=0.03)}

kwargs = {'mean': {'levels': 9, 'vmin':-100, 'vmax':100, 'cmap':cm['mean'], 'extend':'both', **style_kwargs},
          'std':  {'levels': 11, 'vmin':0,    'vmax':50, 'cmap':cm['std'],  'extend':'max',  **style_kwargs}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5))
plt.subplots_adjust(left=0.0125, bottom=0.0125, right=0.99, top=0.935, hspace=0.075, wspace=0.075)

for ix, gcm in enumerate(gcms+['ensemble-mean','ensemble-std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        diff_rcp_hydro_30yr_change['late'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 10:
        diff_rcp_hydro_30yr_change['late'].mean(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 11:
        diff_rcp_hydro_30yr_change['late'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=12)
    
fig.suptitle('RCP85 vs RCP 45: Change in 30yr mean annual evapotransipiration [mm/yr] - late 21st century', fontsize=14, y=0.985)

if figSave:
    plt.savefig(f'NB-HI4_Fig11_annual_mean_change_late_{var}_rcp_diff.png', dpi=300)

## Histogram to show sign and magnitude of the change across the GCMs

In [ ]:
fig, ax = plt.subplots(figsize=(6.5, 4.5))
plt.axhline(y=0, color="black", linestyle="--", linewidth=1)
rcp85_hydro_30yr_diff['late'].mean(dim=['lat','lon']).to_dataframe().drop(columns=['er','rr']).plot.bar(ax=ax)

h,l = ax.get_legend_handles_labels()
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('change signal [mm/yr]')
ax.set_ylim([-300, 600])
ax.set_xlabel('')
ax.set_xticklabels(gcms, rotation = 60)
ax.legend(h[:3],['precipitation','evapotranspiration','runoff'],loc=9)

plt.tight_layout()
if figSave:
    plt.savefig(f'NB-HI4_Fig13_annual_mean_change_late_all_gcm_bar.png', dpi=300)

## Cofficient of variation of future changes for each variables

In [ ]:
# uncertainty
cm = {'cv': cmap}

style_kwargs =  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': dict(orientation="horizontal", pad=0.03)}
kwargs = {'cv': {'levels': 21,'vmin':0, 'vmax':10.0, 'cmap':cm['cv'], 'extend':'max', **style_kwargs}}

var_title = {'PRCP':'Precipitation', 'EVAP':'Evapotranspiration', 'total_runoff':'Runoff'}

fig, axes = plt.subplots(nrows=2, ncols=3, sharex=True, sharey=True, figsize=(6.5, 5.25)) # used to be figsize=(14,11)
plt.subplots_adjust(left=0.050, bottom=0.005, right=0.995, top=0.900, hspace=0.1, wspace=0.1)

mean ={}; std={}; cv={}
for ix, var in enumerate(hydro_flux_vars):
    
    mean[var] = {45: rcp45_hydro_30yr_diff['late'].mean(dim='gcm')[var],
                 85: rcp85_hydro_30yr_diff['late'].mean(dim='gcm')[var]}
    
    std[var]  = {45: rcp45_hydro_30yr_diff['late'].std(dim='gcm')[var],
                 85: rcp85_hydro_30yr_diff['late'].std(dim='gcm')[var]}
    
    cv[var]   = {45: std[var][45]/abs(mean[var][45]),
                 85: std[var][85]/abs(mean[var][85])}
    
    cv[var][45].plot.pcolormesh(ax=axes[0,ix], **kwargs['cv'])
    cv[var][85].plot.pcolormesh(ax=axes[1,ix], **kwargs['cv'])
    axes[0,0].set_ylabel('RCP-4.5', fontsize=11)
    axes[1,0].set_ylabel('RCP-8.5', fontsize=11)
    axes[0,ix].set_title('%s'%var_title[var], fontsize=11)
    
    
fig.suptitle('GCM ensemble CV of flux changes for late-21st century', fontsize=12, y=0.985)
if figSave:
    plt.savefig(f'NB-HI4_Fig12_cv_change_rcp85.png', dpi=300)

In [ ]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature to initialize structure."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
        return value

In [ ]:
# Get std and cv stats over the domain
quantile_list = [0.02,0.1,0.2,0.5,0.8,0.9,0.98]
mean_list = AutoVivification() 
std_list  = AutoVivification()  
cv_list   = AutoVivification() 

for jx, rcp in enumerate([45,85]):
    for ix, var in enumerate(hydro_flux_vars):
    
        #mask out grid boxes with small mean values (< median)
        med = np.nanquantile(mean[var][rcp].values,0.5)
        masked_mean = mean[var][rcp].where(mean[var][rcp]>med)
        masked_std  = std[var][rcp].where(mean[var][rcp]>med)
        masked_cv   = cv[var][rcp].where(mean[var][rcp]>med)
    
        mean_list[var][rcp] = np.nanquantile(mean[var][rcp].values, quantile_list).tolist()
        std_list[var][rcp]  = np.nanquantile(std[var][rcp].values, quantile_list).tolist()
        cv_list[var][rcp]   = np.nanquantile(cv[var][rcp].values,  quantile_list).tolist()

In [ ]:
for rcp in [45,85]:
    print(f'rcp-{rcp}')
    for ix, var in enumerate(hydro_flux_vars):
        print(f'{var}')
        print('quantile:', *quantile_list)
        print('mean    :', *mean_list[var][rcp])
        print('std.dev :', *std_list[var][rcp])
        print('C.V.    :', *cv_list[var][rcp])
        print('\n')

## domain-wide mean change and spread among GCM

In [ ]:
domain_wide_mean = {}
domain_wide_std  = {}
for ix, var in enumerate(hydro_flux_vars):
    
    domain_wide_mean[var] = {45: rcp45_hydro_30yr_diff['late'][var].median(dim=['lon','lat']).mean(dim='gcm').values,
                             85: rcp85_hydro_30yr_diff['late'][var].median(dim=['lon','lat']).mean(dim='gcm').values}
    
    domain_wide_std[var] = {45: rcp45_hydro_30yr_diff['late'][var].median(dim=['lon','lat']).std(dim='gcm').values,
                            85: rcp85_hydro_30yr_diff['late'][var].median(dim=['lon','lat']).std(dim='gcm').values}


In [ ]:
for rcp in [45,85]:
    print(f'rcp-{rcp}')
    for ix, var in enumerate(hydro_flux_vars):
        print(f'{var}')
        print('mean    : ', domain_wide_mean[var][rcp])
        print('std.dev : ', domain_wide_std[var][rcp])
        print('\n')